In [ ]:
]activate ../..

In [ ]:
include("main.jl")
import .LargeRealData: get_params, get_initial_trace

In [ ]:
function printstats(d)
  println("Num entities : ", length(d.entity_strings))
  println("Num verbs    : ", length(d.verb_strings))
  println("Num sentences: ", length(d.sentences_numeric))
end

In [ ]:
printstats(LargeRealData.read_data("sizes/4800.json"))

In [ ]:
filename = "Umass-sub-corpus/pluieTriples_2013_01_06_5.json"
# filename = "06-19/pluieTriples_fgreptest4.json"

In [ ]:
data = LargeRealData.read_data(filename)
printstats(data)

In [ ]:
entpairs = Set(
  ((s.ent1, s.ent2) for s in data.sentences_numeric)
)

In [ ]:
enpair_to_indices = Dict{Tuple{Int, Int}, Set{Int}}(entpair => Set{Int}() for entpair in entpairs)

In [ ]:
for (i, sent) in enumerate(data.sentences_numeric)
  push!(enpair_to_indices[(sent.ent1, sent.ent2)], i)
end

In [ ]:
enpair_to_indices

In [ ]:
enpair_to_lens = Dict(enpair => length(inds) for (enpair, inds) in enpair_to_indices)

In [ ]:
entpairs10 = [pair for (pair, l) in enpair_to_lens if l == 10]

In [ ]:
indices = collect(Iterators.flatten(
      (enpair_to_indices[pair] for pair in entpairs10)
))

In [ ]:
entry(s) = Dict(
[
    "source" => data.entity_strings[s.ent1],
    "dest" => data.entity_strings[s.ent2],
    "depPath" => data.verb_strings[s.verb]
  ]
)

In [ ]:
entries = [entry(s) for s in map(idx -> data.sentences_numeric[idx], indices)]

In [ ]:
using JSON

In [ ]:
new_data = Dict("sentences" => entries)

In [ ]:
jsonstr = JSON.json(new_data)

In [ ]:
open("../data/ten_entpair_per_sentence.json", "w") do f
  write(f, jsonstr)
end

In [ ]:
@time initial_tr = get_initial_trace(sentences, params)
nothing

In [ ]:
@time LargeRealData.run_tracked_splitmerge_inference!(initial_tr, 200, (_, _) -> nothing; log=true, log_freq=100)
nothing

In [ ]:
# 3000 sentences, 4083 entities ---> 200iters/78.87secs

In [ ]:
# 1000 sentences, 1563 entities ---> 200 iters/64.4secs